In [168]:
from bs4 import BeautifulSoup
import requests
import hashlib
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import re
import json
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36")

In [169]:
CACHE_DIR = "cache_html"

if not os.path.exists(CACHE_DIR):
    os.makedirs(CACHE_DIR)

In [170]:
def url_to_filename(url):
    # 把 URL 转成唯一的短文件名
    return hashlib.md5(url.encode('utf-8')).hexdigest() + ".html"

def get_cached_html(url):
    fname = os.path.join(CACHE_DIR, url_to_filename(url))

    if os.path.exists(fname):
        with open(fname, "r", encoding="utf-8") as f:
            return f.read()
    else:
        user_agent = {"User-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"}
        page = requests.get(url, headers=user_agent)
        html = page.content
        
        with open(fname, "w", encoding="utf-8") as f:
            f.write(html.decode('utf-8'))
        return html
    

In [ ]:
base_url = "https://www.amazon.sg"
def get_catergory_url(category):
    """
    Returns the URL for a given category on Amazon Singapore.
    """
    html = get_cached_html(base_url)
    soup = BeautifulSoup(html, "html.parser")
    url = soup.find("a", string=category)['href']
    return f"{base_url}{url}"
url = get_catergory_url("Electronics") #这个类别我已经在爬，可以改成别的类别爬


In [ ]:
def get_featured_categories(index, categories, product_url):
    """
    categories: list of categories to look for
    Returns a list of featured categories on Amazon Singapore.
    """
    if index >= len(categories):
        return
    html = get_cached_html(categories[index])
    time.sleep(1)  # To avoid overwhelming the server
    soup = BeautifulSoup(html, "html.parser")
    pages = soup.find("h1", string=lambda s: s and s.strip() == "Featured categories")
    if not pages:
        get_product_pages(categories[index], product_url)
        #取消注释
        get_featured_categories(index + 1, categories, product_url)
        return
    # print(f"Processing {categories[index]}...")
    pages = pages.find_next("ul").find_all("a", href=True)
    for page in pages:
        href = page['href']
        if not href.startswith("/"):
            continue
        href = f"{base_url}{href}"
        if href not in categories:
            categories.append(href)
    get_featured_categories(index + 1, categories, product_url)

def get_product_pages(category_url, product_url):
    """Returns a list of product pages for a given category URL."""
    print(f"Processing category: {category_url}")
    html = get_cached_html(category_url)
    time.sleep(1)  # To avoid overwhelming the server
    soup = BeautifulSoup(html, "html.parser")
    #<span class="a-size-medium a-color-link a-text-bold">See all results</span>
    #print(f"Processing category: {category_url}")
    current_url = f"{base_url}{soup.find("span", string="See all results").find_parent("a")['href']}"
    #print(f"Current URL: {current_url}")
    product_category = soup.find("h1").text
    while current_url:
    #for page_num in range(1, 2): # Adjust the range for more pages
        soup = BeautifulSoup(get_cached_html(current_url), "html.parser")
        time.sleep(1)
        div_tag = soup.select_one('div.s-main-slot.s-result-list.s-search-results.sg-row')
        for a in div_tag.select('a.a-link-normal.s-line-clamp-4.s-link-style.a-text-normal')[:2]: # Adjust the range for more products
            href = a['href']
            href = f"{base_url}{href}"
            if href not in product_url:
                product_url.append(href)
                get_product_details(see_more_reviews(href), product_category)

        # product_url.extend(
        #     [f"{base_url}{a['href']}" for a in div_tag.select('a.a-link-normal.s-line-clamp-4.s-link-style.a-text-normal')]
        # )
        next_link = soup.select_one('a.s-pagination-item.s-pagination-next.s-pagination-button.s-pagination-button-accessibility.s-pagination-separator')
        if not next_link:
            return
        current_url = f"{base_url}{next_link['href']}"
        # if current_url not in product_url:
        #     product_url.append(current_url)
    product_url = list(set(product_url))  # Remove duplicates
def see_more_reviews(product_url):
    html = get_cached_html(product_url)
    time.sleep(2)  # To avoid overwhelming the server
    soup = BeautifulSoup(html, "html.parser")
    link = soup.find("a", string="See more reviews")
    if not link:
        return product_url
    see_more_reviews_url = link['href']
    return f"{base_url}{see_more_reviews_url}"

# 取消注释
# for url in product_url:
#     url = see_more_reviews(url)
#     print(f"Updated URL: {url}")
#     get_product_details(url)


driver = webdriver.Chrome(options=options)
driver.get("https://www.amazon.sg/product-reviews/B0C7GGYS7M/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews")
#确保只登录一次
email_box = driver.find_element(By.ID, "ap_email_login")
email_box.send_keys("your phone number here") #一定要改
driver.find_element(By.CSS_SELECTOR, '#continue input[type="submit"]').click()
#driver.find_element(By.ID, "continue").click()
time.sleep(2)
password_box = driver.find_element(By.ID, "ap_password")
password_box.send_keys("your password here") #一定要改
driver.find_element(By.ID, "signInSubmit").click()
time.sleep(2)
product_details = []
def get_product_details(review_url, catergory):
    # html = get_cached_html(review_url)
    # time.sleep(1)  # To avoid overwhelming the server
    # soup = BeautifulSoup(html, "html.parser")
    # print(f"Processing {review_url}...")
    # with open("page2.html", "w", encoding="utf-8") as f:
    #     f.write(html)
    driver.get(review_url)
    time.sleep(2)  # Wait for the page to load
    while True:
        print(f"Processing {review_url}...")
    #for page_num in range(1, 3):  # Adjust the range for more pages
        html = driver.page_source
        # with open("page2.html", "w", encoding="utf-8") as f:
        #     f.write(html)
        soup = BeautifulSoup(html, "html.parser")
        #reviews = soup.css('li[data-hook="review"]')
        reviews = soup.select('li[data-hook="review"]')
        #reviews = soup.find_all("li", {'data-hook': 'review'})###???!!!
        #print(f"Found {len(reviews)} reviews on {review_url}")
        for review in reviews:
            reviewerID = review['id']
            reviewerName = review.find("span", class_="a-profile-name").get_text(strip=True)
            #print(review.find('i', attrs={'data-hook': re.compile('star')}).)
            #print(review.find('i', attrs={'data-hook': 'cmps-review-star-rating'}).get_text(strip=True))
            overall = float(
                review.find('i', attrs={'data-hook': re.compile('star')})
                    .find('span', class_='a-icon-alt')
                    .get_text(strip=True)
                    .split()[0]
            )
            reviewTime = review.find('span', {'data-hook': 'review-date'}).text.split(' on ')[-1]
            reviewText = review.find('span', attrs={'data-hook': 'review-body'}).get_text(separator=' ', strip=True)
            span = review.find('span', attrs={'data-hook': 'helpful-vote-statement'})
            if span:
                text = span.get_text(strip=True).lower()
                if 'one person found' in text:
                    helpful = 1
                else:
                    match = re.search(r'\d+', text)
                    helpful = int(match.group()) if match else 0
            else:
                helpful = 0
            product_details.append({
                "reviewerID": reviewerID,
                "reviewerName": reviewerName,
                "helpful": [helpful,0],
                "reviewText": reviewText,
                "overall": overall,
                "reviewTime": reviewTime,
                "catergory": catergory
            })
            if len(product_details) % 1000 == 0:
                with open("product_details.json", "a", encoding="utf-8") as f:
                    for item in product_details:
                        json.dump(item, f, ensure_ascii=False)
                        f.write("\n")
                product_details.clear()
        flag = False
        for a in soup.find_all('a', href=True):
            if a.get_text(strip=True).startswith('Next page'):
                review_url = a['href']
                flag = True
                break
        if flag:
            review_url = base_url + review_url
        else:
            return
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, "li.a-last a")
            if 'disabled' in next_button.get_attribute('class'):
                break
            next_button.click()
            time.sleep(3)  # 等待页面加载
        except Exception as e:
            break


In [173]:
featured_categories = [url]
product_url = []
get_featured_categories(0, featured_categories, product_url)
#get_product_details()  # Call the function to populate
#get_product_details(see_more_reviews("https://www.amazon.sg/Smart-Watch-Women-Smartwatches-Notification/dp/B0CZDZHGWJ/ref=sr_1_2?dib=eyJ2IjoiMSJ9.28r7aBVvB5vOOhjcUftHNy-vy4-PGtan-PabGuVo5kljPblR5eu6hafG9DLShsFuPad_ZFRRP4eyKGq2NtkdwqkqE6rDGVrD5cIrR2hXneSMZ2sIRmfHfl3koHoIoAgHOKoR3MV48Wqhhydqi63NENiM5LRCML-BDeEzrBBTn1utr8v9g-BmN8Kh-Vm9uGicn3PpckA9xD_UJyiKCDNctwDtIRXbXbTqHXzgnza-f-mtVfsd3jNt3lPx1EA46Kza1LK0fDgh5o4YLv8iZ5H3E7bTrAmBIOSefV_zaHW-o10.NmFPZtMtM4ueRvEdjFdx970DaG6PZkpmoeHi90yzOzs&dib_tag=se&qid=1751638016&s=electronics&sr=1-2&th=1"))  # Example URL
#print(product_details)
product_details

Processing category: https://www.amazon.sg/b/?_encoding=UTF8&node=6436180051&bbn=6314449051&ref_=Oct_d_odnav_6314449051&pd_rd_w=reBub&content-id=amzn1.sym.b0a21f0e-ffd6-4574-ba2e-f72903cf4d47&pf_rd_p=b0a21f0e-ffd6-4574-ba2e-f72903cf4d47&pf_rd_r=5B4GZ3G8MPWK8HZF0F3C&pd_rd_wg=m66kM&pd_rd_r=6cb599c3-5092-4ab8-97ba-b72a9ebbf38c
Processing category: https://www.amazon.sg/b/?_encoding=UTF8&node=6436180051&bbn=6314449051&ref_=Oct_d_odnav_6314449051&pd_rd_w=reBub&content-id=amzn1.sym.b0a21f0e-ffd6-4574-ba2e-f72903cf4d47&pf_rd_p=b0a21f0e-ffd6-4574-ba2e-f72903cf4d47&pf_rd_r=5B4GZ3G8MPWK8HZF0F3C&pd_rd_wg=m66kM&pd_rd_r=6cb599c3-5092-4ab8-97ba-b72a9ebbf38c
Current URL: https://www.amazon.sg/s?i=electronics&bbn=6314449051&rh=n%3A6436180051&s=popularity-rank&fs=true&ref=lp_6436180051_sar
{"reviewerID": "R331X05IFVHYYK", "reviewerName": "James Wong", "helpful": [0, 0], "reviewText": "Very responsive and beautiful watch. Battery lasts about 10 days with normal usage, some sports and some walking. Good 

[{'reviewerID': 'R331X05IFVHYYK',
  'reviewerName': 'James Wong',
  'helpful': [0, 0],
  'reviewText': 'Very responsive and beautiful watch. Battery lasts about 10 days with normal usage, some sports and some walking. Good record of my sleeping timings.',
  'overall': 5.0,
  'reviewTime': '4 March 2025',
  'catergory': 'Smartwatches'},
 {'reviewerID': 'R1I167K9K0HETG',
  'reviewerName': 'Nuzaibah S',
  'helpful': [0, 0],
  'reviewText': 'Looks superb n stylish',
  'overall': 5.0,
  'reviewTime': '11 February 2025',
  'catergory': 'Smartwatches'},
 {'reviewerID': 'R2IELXS44ADOTO',
  'reviewerName': 'Nurdiana Roslee',
  'helpful': [0, 0],
  'reviewText': 'It is awesome, I need to charge it once a week, as I use it almost always for exercises and work.',
  'overall': 4.0,
  'reviewTime': '21 August 2024',
  'catergory': 'Smartwatches'},
 {'reviewerID': 'R2G91QWKCBJLY7',
  'reviewerName': 'Punitha',
  'helpful': [0, 0],
  'reviewText': 'More than expected. Worth the money!',
  'overall': 5